In [2]:
# Install Required Libraries in Terminal
# pip install transformers datasets accelerate peft bitsandbytes torch openpyxl bert_score
# Done

In [5]:
pip install huggingface_hub transformers datasets accelerate peft bitsandbytes torch openpyxl bert_score

Note: you may need to restart the kernel to use updated packages.


In [6]:
# disables tokenizer thread parallelism, 
# telling Hugging Face to run tokenization single-threaded, avoiding the risk of threading issues across forked processes.
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Login into hugging face to get access to LLM

In [ ]:
from huggingface_hub import login
login("") # USE YOUR OWN TOKEN

## Load the model

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM  # Import tokenizer and model loader
import torch  # PyTorch for handling tensors and model inference
import pandas as pd  # Pandas for reading and processing the dataset
import bert_score  # BERTScore for evaluating model-generated responses

# ========== 1. Prevent model from being downloaded repeatedly ==========
model_id = "meta-llama/Llama-3.2-3B-Instruct"  # Model name
cache_dir = "/workspace/model_cache"  # Set cache directory to store model locally

# Ensure the cache directory exists
import os
os.makedirs(cache_dir, exist_ok=True)

# Load tokenizer from cache (if available); otherwise, download and store it
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

# Load model with reduced memory usage and automatic GPU allocation
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Use bfloat16 to save memory while maintaining precision
    device_map="auto",  # Automatically assigns model to GPU (if available)
    cache_dir=cache_dir  # Store model in cache to avoid re-downloading
)

print("Model successfully loaded from cache:", cache_dir)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Model successfully loaded from cache: /workspace/model_cache


# FROM HERE, 
# after reload the NEW & Trained model, 
# you can re-run the codes below till the end of BLEU

In [19]:
# ========== 2. Load dataset ==========
data_path = "VerifiedQ_A.xlsx"  # Path to dataset file
df = pd.read_excel(data_path)  # Load the dataset into a DataFrame
print(df.head(10))  # Print the first few rows to verify correct loading


                      AnswerID    Unnamed: 2                    Unnamed: 3  \
0        ADAM_0002818_Sec1.txt  3-Incomplete         ADAM_0002818_Sec1.txt   
1        ADAM_0002818_Sec2.txt  3-Incomplete         ADAM_0002818_Sec2.txt   
2        GARD_0004450_Sec1.txt  3-Incomplete         GARD_0004450_Sec1.txt   
3        GARD_0004450_Sec2.txt  3-Incomplete         GARD_0004450_Sec2.txt   
4        GARD_0004450_Sec3.txt  3-Incomplete         GARD_0004450_Sec3.txt   
5         GHR_0000804_Sec1.txt  3-Incomplete          GHR_0000804_Sec1.txt   
6         GHR_0000804_Sec3.txt  3-Incomplete          GHR_0000804_Sec3.txt   
7         GHR_0000804_Sec4.txt  3-Incomplete          GHR_0000804_Sec4.txt   
8  MPlusDrugs_0001309_Sec2.txt  3-Incomplete   MPlusDrugs_0001309_Sec2.txt   
9  MPlusDrugs_0001309_Sec5.txt  3-Incomplete   MPlusDrugs_0001309_Sec5.txt   

                                            Question  \
0                    What is (are) Noonan syndrome ?   
1                      What c

In [20]:
# ========== 3. Generate answers using Llama 3.2 ==========

# generate_answer() with Post-Trim Cleanup
# ISSUE: Llama always generate its model answer along with the question itself, unnecessarily consuming the max_token space.
# ISSUE: it would be problematic if the question is very long!!!

#Revised generate_answer() with Improved Echo Trimming
def generate_answer(question):
    """Generates a clean model response from LLaMA using a direct instruction prompt."""

    if not isinstance(question, str):
        return "Skipped"

    # Instruction-style prompt to guide model toward answering
    prompt = f"Answer concisely:\n\n{question}\n\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=256,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Strip everything before "Answer:" if model includes it in output
    if "Answer:" in decoded:
        decoded = decoded.split("Answer:")[-1].strip()

    return decoded


# Apply model to generate answers only for valid text questions
df["Model_Answer"] = df["Question"].apply(generate_answer)

# Print skipped questions for debugging
skipped_count = (df["Model_Answer"] == "Skipped").sum()
print(f"⚠️ {skipped_count} questions were skipped due to invalid format.")

⚠️ 0 questions were skipped due to invalid format.


Save the data to make sure all evaluations are based on the same data

In [21]:
df.to_excel("generated_answers.xlsx", index=False)
print("DataFrame saved as generated_answers.xlsx")

DataFrame saved as generated_answers.xlsx


## For BertScore

In [22]:
import pandas as pd

# Load the saved Excel file
df = pd.read_excel("generated_answers.xlsx")

print("✅ DataFrame reloaded successfully!")
print(df.head())  # Display the first few rows to verify

✅ DataFrame reloaded successfully!
                AnswerID    Unnamed: 2              Unnamed: 3  \
0  ADAM_0002818_Sec1.txt  3-Incomplete   ADAM_0002818_Sec1.txt   
1  ADAM_0002818_Sec2.txt  3-Incomplete   ADAM_0002818_Sec2.txt   
2  GARD_0004450_Sec1.txt  3-Incomplete   GARD_0004450_Sec1.txt   
3  GARD_0004450_Sec2.txt  3-Incomplete   GARD_0004450_Sec2.txt   
4  GARD_0004450_Sec3.txt  3-Incomplete   GARD_0004450_Sec3.txt   

                                            Question  \
0                    What is (are) Noonan syndrome ?   
1                      What causes Noonan syndrome ?   
2  What is (are) Noonan syndrome ? (Also called: ...   
3  What are the symptoms of Noonan syndrome ? (Al...   
4  Is Noonan syndrome inherited ? (Also called: M...   

                                              Answer  \
0  Noonan syndrome is a disease that can be passe...   
1  Noonan syndrome is linked to defects in severa...   
2  Noonan syndrome is a genetic disorder that cau...   
3  What

In [23]:
# ========== 4. Compute BERTScore ==========
import bert_score
# Filter out skipped answers before evaluation
valid_rows = df[df["Model_Answer"] != "Skipped"].copy()  # Keep only valid responses

# Ensure that valid_rows has aligned Model Answers and Reference Answers
P, R, F1 = bert_score.score(
    valid_rows["Model_Answer"].tolist(),  # Only valid generated answers
    valid_rows["Answer"].tolist(),  # Corresponding reference answers
    lang="en"  # Specify language as English
)

# Store computed BERTScore metrics in the filtered DataFrame
valid_rows["BERT_Precision"] = P.tolist()
valid_rows["BERT_Recall"] = R.tolist()
valid_rows["BERT_F1"] = F1.tolist()

# ========== 5. Save results ==========
valid_rows.to_csv("bert_score_results.csv", index=False)  # Save only valid results
print(f"✅ Computation completed. {len(valid_rows)} valid responses evaluated.")
print("Results saved as `bert_score_results.csv`")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Computation completed. 150 valid responses evaluated.
Results saved as `bert_score_results.csv`


## For DeepEval Scores

In [1]:
!pip install --upgrade pydantic typing_extensions deepeval

I RESTART the kernel due to unexpected issue.

This time, I will simply load the saved df file generated_answers.xlsx

if you see some red sign from BertScore, that is because I restart the kernel.

In [3]:
import pandas as pd

# Load the saved Excel file
df = pd.read_excel("generated_answers.xlsx")

print("✅ DataFrame reloaded successfully!")
print(df.head())  # Display the first few rows to verify

✅ DataFrame reloaded successfully!
                AnswerID    Unnamed: 2              Unnamed: 3  \
0  ADAM_0002818_Sec1.txt  3-Incomplete   ADAM_0002818_Sec1.txt   
1  ADAM_0002818_Sec2.txt  3-Incomplete   ADAM_0002818_Sec2.txt   
2  GARD_0004450_Sec1.txt  3-Incomplete   GARD_0004450_Sec1.txt   
3  GARD_0004450_Sec2.txt  3-Incomplete   GARD_0004450_Sec2.txt   
4  GARD_0004450_Sec3.txt  3-Incomplete   GARD_0004450_Sec3.txt   

                                            Question  \
0                    What is (are) Noonan syndrome ?   
1                      What causes Noonan syndrome ?   
2  What is (are) Noonan syndrome ? (Also called: ...   
3  What are the symptoms of Noonan syndrome ? (Al...   
4  Is Noonan syndrome inherited ? (Also called: M...   

                                              Answer  \
0  Noonan syndrome is a disease that can be passe...   
1  Noonan syndrome is linked to defects in severa...   
2  Noonan syndrome is a genetic disorder that cau...   
3  What

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "" # USE YOUR OWN TOKEN!

In [5]:
# ========== 6. Compute DeepEval Correctness ==========
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

# Initialize metric
correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the expected output. Provide 1 for correct and 0 for incorrect.",
    evaluation_steps=[
        "Check if 'actual output' contradicts 'expected output'",
        "Only penalize omission of important details"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model="gpt-4o-mini"  # or "gpt-4", "gpt-3.5-turbo", etc.
)

# Filter valid rows again
valid_rows = df[
    (df["Model_Answer"].notna()) &
    (df["Answer"].notna()) &
    (df["Question"].notna()) &
    (df["Model_Answer"] != "Skipped")
].copy()

# Prepare to collect scores
correctness_scores = []

for i in range(len(valid_rows)):
    try:
        question = valid_rows["Question"].iloc[i]
        model_answer = valid_rows["Model_Answer"].iloc[i]
        reference_answer = valid_rows["Answer"].iloc[i]

        print(f"\n--- [Row {i}] ---")
        print("Question:", question[:100])
        print("Model_Answer:", model_answer[:100])
        print("Reference Answer:", reference_answer[:100])

        test_case = LLMTestCase(
            input=question,
            actual_output=model_answer,
            context=[reference_answer]
        )

        result = correctness_metric.measure(test_case)

        # ✅ THIS is the actual correctness score
        score = float(result)
        correctness_scores.append(score)

        print(f"[{i}] 🎯 Correctness Score (raw): {score}")

    except Exception as e:
        print(f"[{i}] ⚠️ Error: {e}")
        correctness_scores.append(0.0)
        
# Assign the scores
valid_rows["DeepEval_Correctness"] = correctness_scores

# ========== 7. Save Final Results ==========
valid_rows.to_excel("final_eval_results.xlsx", index=False)
print("✅ Final results saved to final_eval_results.xlsx")

Output()


--- [Row 0] ---
Question: What is (are) Noonan syndrome ?
Model_Answer: Noonan syndrome is a disease that can be passed down through families (inherited). It causes many pa
Reference Answer: Noonan syndrome is a disease that can be passed down through families (inherited). It causes many pa


Output()

[0] 🎯 Correctness Score (raw): 0.7810623892820897

--- [Row 1] ---
Question: What causes Noonan syndrome ?
Model_Answer: Noonan syndrome is linked to defects in several genes. Certain proteins involved in growth and devel
Reference Answer: Noonan syndrome is linked to defects in several genes. Certain proteins involved in growth and devel


Output()

...

Output()

[147] 🎯 Correctness Score (raw): 0.7782141599535661

--- [Row 148] ---
Question: How to diagnose Autoimmune disorders ?
Model_Answer: The health care provider will do a physical exam. Signs depend on the type of disease.  Tests that m
Reference Answer: The health care provider will do a physical exam. Signs depend on the type of disease.  Tests that m


Output()

[148] 🎯 Correctness Score (raw): 0.7344664206323175

--- [Row 149] ---
Question: What should I know about storage and disposal of Glimepiride ?
Model_Answer: Keep this medication in the container it came in, tightly closed, and out of reach of children. Stor
Reference Answer: Keep this medication in the container it came in, tightly closed, and out of reach of children. Stor


[149] 🎯 Correctness Score (raw): 0.8077238220745988
✅ Final results saved to final_eval_results.xlsx


## For BLEU Score

In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd

# Load the saved Excel file
df = pd.read_excel("generated_answers.xlsx")

print("✅ DataFrame reloaded successfully!")
print(df.head())  # Display the first few rows to verify

✅ DataFrame reloaded successfully!
                AnswerID    Unnamed: 2              Unnamed: 3  \
0  ADAM_0002818_Sec1.txt  3-Incomplete   ADAM_0002818_Sec1.txt   
1  ADAM_0002818_Sec2.txt  3-Incomplete   ADAM_0002818_Sec2.txt   
2  GARD_0004450_Sec1.txt  3-Incomplete   GARD_0004450_Sec1.txt   
3  GARD_0004450_Sec2.txt  3-Incomplete   GARD_0004450_Sec2.txt   
4  GARD_0004450_Sec3.txt  3-Incomplete   GARD_0004450_Sec3.txt   

                                            Question  \
0                    What is (are) Noonan syndrome ?   
1                      What causes Noonan syndrome ?   
2  What is (are) Noonan syndrome ? (Also called: ...   
3  What are the symptoms of Noonan syndrome ? (Al...   
4  Is Noonan syndrome inherited ? (Also called: M...   

                                              Answer  \
0  Noonan syndrome is a disease that can be passe...   
1  Noonan syndrome is linked to defects in severa...   
2  Noonan syndrome is a genetic disorder that cau...   
3  What

In [8]:
# Filter valid rows again
valid_rows = df[
    (df["Model_Answer"].notna()) &
    (df["Answer"].notna()) &
    (df["Question"].notna()) &
    (df["Model_Answer"] != "Skipped")
].copy()

print(valid_rows.head())

                AnswerID    Unnamed: 2              Unnamed: 3  \
0  ADAM_0002818_Sec1.txt  3-Incomplete   ADAM_0002818_Sec1.txt   
1  ADAM_0002818_Sec2.txt  3-Incomplete   ADAM_0002818_Sec2.txt   
2  GARD_0004450_Sec1.txt  3-Incomplete   GARD_0004450_Sec1.txt   
3  GARD_0004450_Sec2.txt  3-Incomplete   GARD_0004450_Sec2.txt   
4  GARD_0004450_Sec3.txt  3-Incomplete   GARD_0004450_Sec3.txt   

                                            Question  \
0                    What is (are) Noonan syndrome ?   
1                      What causes Noonan syndrome ?   
2  What is (are) Noonan syndrome ? (Also called: ...   
3  What are the symptoms of Noonan syndrome ? (Al...   
4  Is Noonan syndrome inherited ? (Also called: M...   

                                              Answer  \
0  Noonan syndrome is a disease that can be passe...   
1  Noonan syndrome is linked to defects in severa...   
2  Noonan syndrome is a genetic disorder that cau...   
3  What are the signs and symptoms of Noon

In [9]:
# ========== 8. Compute BLEU Score (No punkt needed) ==========
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smoothie = SmoothingFunction().method4  # Smoothing for short sentences

# BLEU score calculation helper using simple whitespace split
def compute_bleu(reference, hypothesis):
    if not reference or not hypothesis:
        return 0.0
    try:
        reference_tokens = reference.split()
        hypothesis_tokens = hypothesis.split()

        return sentence_bleu(
            [reference_tokens], hypothesis_tokens,
            weights=(0.25, 0.25, 0.25, 0.25),  # BLEU-4
            smoothing_function=smoothie
        )
    except Exception as e:
        print(f"⚠️ BLEU computation error: {e}")
        return 0.0

# Store BLEU scores
bleu_scores = []

# Loop through rows and print BLEU computation info
for i, row in valid_rows.iterrows():
    reference = row["Answer"]
    hypothesis = row["Model_Answer"]
    score = compute_bleu(reference, hypothesis)
    bleu_scores.append(score)

# Assign scores to DataFrame
valid_rows["BLEU_Score"] = bleu_scores

# ========== 9. Save BLEU Results ==========
# Create a subset of columns to save
results_to_save = valid_rows[[
    "Question",
    "Answer",           # Reference Answer
    "Model_Answer",
    "BLEU_Score"
]].copy()

# Rename for clarity
results_to_save.rename(columns={
    "Answer": "Reference Answer"
}, inplace=True)

# Save to CSV
results_to_save.to_csv("bleu_score_results.csv", index=False)
print("📄 BLEU results saved to bleu_score_results.csv")

📄 BLEU results saved to bleu_score_results.csv


# THE END OF RE-RUN SECTION

## Tune the model (LoRA)

In [9]:
# ===============================
# LoRA Fine-Tuning Script for LLaMA 3.2 3B/7B on Medquad
# ===============================

import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

# ===============================
# Step 1: Define Paths & Model ID
# ===============================
model_id = "meta-llama/Llama-3.2-3B-Instruct"  # You can also use the 3B model here
cache_dir = "/workspace/model_cache"
data_path = "VerifiedQ_A.xlsx"
output_dir = "./llama3_lora_medquad"

os.makedirs(cache_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [10]:
# ===============================
# Step 2: Load Dataset
# ===============================
df = pd.read_excel(data_path)
df = df.dropna(subset=["Question", "Answer"])

def format_prompt(example):
    return {
        "text": f"### Question:\n{example['Question']}\n\n### Answer:\n{example['Answer']}"
    }

raw_dataset = Dataset.from_pandas(df)
formatted_dataset = raw_dataset.map(format_prompt)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [11]:
# ===============================
# Step 3: Tokenizer
# ===============================
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(
        example["text"], truncation=True, padding="max_length", max_length=512
    )

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [12]:
# ===============================
# Step 4: Load Model (Full Precision) + Inject LoRA
# ===============================
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    cache_dir=cache_dir
)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# ===============================
# Step 5: Training Configuration (Updated)
# ===============================
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,  # Run optimizer step after every batch
    num_train_epochs=10,  # More epochs for small dataset
    learning_rate=2e-4,
    bf16=True,
    logging_dir=f"{output_dir}/logs",
    save_strategy="epoch",
    report_to="none",
    logging_steps=1,  # Optional: more frequent logging
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [14]:
# ===============================
# Step 6: Trainer
# ===============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/tmp/ipykernel_1999/1826885073.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,2.074400
2,2.246700
3,1.769500
4,2.157400
5,1.840300
6,1.949000
7,2.005600
8,1.498400
9,1.769200
10,1.955400


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

TrainOutput(global_step=380, training_loss=1.1147158563921327, metrics={'train_runtime': 124.7368, 'train_samples_per_second': 12.025, 'train_steps_per_second': 3.046, 'total_flos': 1.3073345150976e+16, 'train_loss': 1.1147158563921327, 'epoch': 10.0})

In [15]:
# ===============================
# Step 7: Save Model & Tokenizer
# ===============================
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Fine-tuning complete! Model saved to {output_dir}")

Fine-tuning complete! Model saved to ./llama3_lora_medquad


## Load the NEW LoRA model

In [16]:
# ===============================
# Step 8: Load Fine-Tuned LoRA Model for Evaluation
# ===============================
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import os

# Define the model directory where LoRA was saved
lora_model_dir = "./llama3_lora_medquad"
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
cache_dir = "/workspace/model_cache"

# Load base LLaMA model (required for LoRA base)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir=cache_dir
)

# Load the LoRA adapters
model = PeftModel.from_pretrained(base_model, lora_model_dir)
model.eval()  # Set to eval mode

# Load tokenizer from fine-tuned folder (same tokenizer)
tokenizer = AutoTokenizer.from_pretrained(lora_model_dir)

print("Fine-tuned LoRA model and tokenizer successfully loaded!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fine-tuned LoRA model and tokenizer successfully loaded!


In [25]:
# ===============================
# 🔍 Optional: Confirm it works with a test question
# ===============================
prompt = "Answer concisely:\n\nWhat are the symptoms of diabetes?\n\nAnswer:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=256,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🧪 Sample Output:\n", response)

🧪 Sample Output:
 Answer concisely:

What are the symptoms of diabetes?

Answer: 
Diabetes symptoms can vary depending on the type and severity of the condition. Common symptoms include:

* Increased thirst and hunger
* Frequent urination
* Fatigue
* Blurred vision
* Slow healing of cuts and wounds
* Tingling or numbness in hands and feet
* Unexplained weight loss
* Recurring skin, gum, or bladder infections

In type 1 diabetes, symptoms appear suddenly and can include:
* Severe vomiting
* Abdominal pain
* Diarrhea
* Nausea
* Headache

In type 2 diabetes, symptoms often develop gradually and may include:
* Increased appetite
* Weight gain
* Increased urination
* Fatigue
* Blurred vision

Note: Some people with diabetes may not exhibit any symptoms at all, especially in the early stages of the condition. Regular blood glucose monitoring and check-ups with a healthcare provider can help detect diabetes before symptoms appear.


## Fine-Tuning

In [8]:
# ===============================
# Clean Fine-Tuning Script for LLaMA 3.2 3B (No Benchmarking, From Scratch)
# ===============================

import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)

# ===============================
# Step 1: Load model
# ===============================
model_id = "meta-llama/Llama-3.2-3B-Instruct"  # Model name
cache_dir = "/workspace/model_cache"  # Set cache directory to store model locally

# Ensure the cache directory exists
os.makedirs(cache_dir, exist_ok=True)

# Load tokenizer from cache (if available); otherwise, download and store it
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Load model with reduced memory usage and automatic GPU allocation
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir=cache_dir
)

print("✅ Model successfully loaded from cache:", cache_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model successfully loaded from cache: /workspace/model_cache


In [10]:
# ===============================
# Step 2: Define Dataset
# ===============================
data_path = "VerifiedQ_A.xlsx"
output_dir = "./llama3_finetune_medquad"
os.makedirs(output_dir, exist_ok=True)

# ===============================
# Step 3: Load Dataset
# ===============================
df = pd.read_excel(data_path)
df = df.dropna(subset=["Question", "Answer"])
print(df.head()) # Quick check

def format_prompt(example): # mimics instruction-following prompts which models like LLaMA are tuned for
    return {
        "text": f"### Question:\n{example['Question']}\n\n### Answer:\n{example['Answer']}"
    }

raw_dataset = Dataset.from_pandas(df) # Convert DataFrame to Hugging Face Dataset
formatted_dataset = raw_dataset.map(format_prompt) # Apply Prompt Formatting

                AnswerID    Unnamed: 2              Unnamed: 3  \
0  ADAM_0002818_Sec1.txt  3-Incomplete   ADAM_0002818_Sec1.txt   
1  ADAM_0002818_Sec2.txt  3-Incomplete   ADAM_0002818_Sec2.txt   
2  GARD_0004450_Sec1.txt  3-Incomplete   GARD_0004450_Sec1.txt   
3  GARD_0004450_Sec2.txt  3-Incomplete   GARD_0004450_Sec2.txt   
4  GARD_0004450_Sec3.txt  3-Incomplete   GARD_0004450_Sec3.txt   

                                            Question  \
0                    What is (are) Noonan syndrome ?   
1                      What causes Noonan syndrome ?   
2  What is (are) Noonan syndrome ? (Also called: ...   
3  What are the symptoms of Noonan syndrome ? (Al...   
4  Is Noonan syndrome inherited ? (Also called: M...   

                                              Answer  
0  Noonan syndrome is a disease that can be passe...  
1  Noonan syndrome is linked to defects in severa...  
2  Noonan syndrome is a genetic disorder that cau...  
3  What are the signs and symptoms of Noonan s

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [13]:
# ===============================
# Step 4: Tokenize Dataset
# ===============================
# Tokenize the formatted text prompts with max length and padding
# Returns token ids and attention masks used during training

def tokenize(example):
    return tokenizer(
        example["text"], truncation=True, padding="max_length", max_length=512
    )

# Apply tokenization to the entire dataset in batches for speed
tokenized_dataset = formatted_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [14]:
# ===============================
# Step 5: Training Configuration
# ===============================
# Define all hyperparameters and runtime configs for training
training_args = TrainingArguments(
    output_dir=output_dir,  # Folder to save model checkpoints
    per_device_train_batch_size=4,  # Batch size per GPU
    gradient_accumulation_steps=1,  # Gradient steps before backward/update
    num_train_epochs=10,  # Number of passes over the full training data
    learning_rate=5e-5,  # Initial learning rate for AdamW optimizer
    bf16=True,  # Use bfloat16 if supported by GPU (e.g., A100)
    logging_dir=f"{output_dir}/logs",  # Save training logs here
    save_strategy="epoch",  # Save model once per epoch
    report_to="none",  # Disable integration with wandb/huggingface
    logging_steps=10,  # Log loss etc. every N steps
    save_total_limit=2,  # Keep only last 2 checkpoints to save disk
    warmup_steps=100  # Gradually ramp up LR to avoid instability
)

# Create a data collator that dynamically pads batches during training
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [15]:
# ===============================
# Step 6: Trainer
# ===============================
# The Trainer class handles the training loop and saves the best model
trainer = Trainer(
    model=model,  # The model to fine-tune
    args=training_args,  # Training configuration
    train_dataset=tokenized_dataset,  # Input data
    tokenizer=tokenizer,  # Tokenizer for saving and decoding
    data_collator=data_collator  # Handles padding during training
)

# Start training
trainer.train()

/tmp/ipykernel_2763/311759560.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.970600
20,1.684600
30,1.512800
40,1.356800
50,1.167300
60,1.076800
70,1.032000
80,0.855400
90,0.574800
100,0.506000


TrainOutput(global_step=380, training_loss=0.4065922520662609, metrics={'train_runtime': 545.3352, 'train_samples_per_second': 2.751, 'train_steps_per_second': 0.697, 'total_flos': 1.2988787982336e+16, 'train_loss': 0.4065922520662609, 'epoch': 10.0})

In [16]:
# ===============================
# Step 7: Save Model & Tokenizer
# ===============================
# Save the final fine-tuned model and tokenizer for inference or reuse
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\n✅ Full fine-tuning complete! Model saved to {output_dir}")


✅ Full fine-tuning complete! Model saved to ./llama3_finetune_medquad


## Load the NEW Fine-Tuned Model

In [18]:
# ===============================
# Step 8: Load Fine-Tuned Full Model for Inference
# ===============================
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

# Define paths to the saved model and cache
finetuned_model_dir = "./llama3_finetune_medquad"  # Your saved model directory
cache_dir = "/workspace/model_cache"  # Cache path

# Load the fine-tuned model
model = AutoModelForCausalLM.from_pretrained(
    finetuned_model_dir,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir=cache_dir
)
model.eval()  # Set to eval mode to disable dropout etc.

# Load the tokenizer (from same fine-tuned folder)
tokenizer = AutoTokenizer.from_pretrained(finetuned_model_dir)
tokenizer.pad_token = tokenizer.eos_token  # Ensure pad token is defined

print("✅ Fine-tuned full model and tokenizer successfully loaded!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Fine-tuned full model and tokenizer successfully loaded!


## Now go back the top and START from the "# FROM HERE, after reload the NEW & Trained model, you can re-run the codes below till the end of BLEU"